In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
all_vacancy_list = []
vacancy_dict = {}
i = 1

In [3]:
while (True):
    webLink = f"https://busy.az/professions/software-developer?page={i}"
    response = requests.get(webLink)

    if(response.status_code == 200):
        soup = BeautifulSoup(response.content, 'lxml',from_encoding='utf-8')
        all_vacancy_cards = soup.find_all('a', class_="job-listing with-apply-button")

        for card in all_vacancy_cards:
            link_to_vacancy_detail = card.get("href")
            vacancy_name = card.find("h3", class_="job-listing-title").get_text()
            company_name = card.find("li").get_text().strip()

            vacancy_dict = {
                    "Name": vacancy_name,
                    "Url to Vacancy": link_to_vacancy_detail,
                    "Company": company_name
            }
            
            response_to_vacancy_detail = requests.get(link_to_vacancy_detail)
            
            if(response_to_vacancy_detail.status_code==200):
                vacancy_details = BeautifulSoup(response_to_vacancy_detail.content, 'lxml',from_encoding='utf-8').find("div", class_="job-overview-inner")

                if vacancy_details.find("span", string="Yer"):
                    location = vacancy_details.find("span", string="Yer").find_next_sibling("h5").text.strip()
                    vacancy_dict["Location"] = location
                else:
                    vacancy_dict["Location"] = ""

                if vacancy_details.find("span", string="Məşğulluq növü"):
                    type_of_employment = vacancy_details.find("span", string="Məşğulluq növü").find_next_sibling("h5").text.strip()
                    vacancy_dict["Type of Employment"] = type_of_employment
                else:
                    vacancy_dict["Type of Employment"] = ""

                if vacancy_details.find("span", string="Yaş"):
                    age_category = vacancy_details.find("span", string="Yaş").find_next_sibling("h5").text.strip()  
                    vacancy_dict["Age category"] = age_category
                else:
                    vacancy_dict["Age category"] = ""

                if vacancy_details.find("span", string="Təhsil"):
                    education_degree = vacancy_details.find("span", string="Təhsil").find_next_sibling("h5").text.strip()
                    vacancy_dict["Education degree"] = education_degree
                else:
                    vacancy_dict["Education degree"] = ""

                if vacancy_details.find("span", string="Maaş"):
                    salary = vacancy_details.find("span", string="Maaş").find_next_sibling("h5").text.strip()
                    vacancy_dict["Salary"] = salary
                else:
                    vacancy_dict["Salary"] = ""

                if vacancy_details.find("span", string="Elanın qoyulma tarixi"):
                    posting_date = vacancy_details.find("span", string="Elanın qoyulma tarixi").find_next_sibling("h5").text.strip()
                    vacancy_dict["Posting Date"] = posting_date
                else:
                    vacancy_dict["Posting Date"] = ""

                if vacancy_details.find("span", string="Son müraciət tarixi"):
                    deadline_date = vacancy_details.find("span", string="Son müraciət tarixi").find_next_sibling("h5").text.strip()
                    vacancy_dict["Deadline Date"] = deadline_date
                else:
                    vacancy_dict["Deadline Date"] = ""

                if vacancy_details.findAll("div", class_="task-tags"):
                    tags_container = vacancy_details.findAll("div", class_="task-tags")

                    if len(tags_container) > 0:
                        all_tags = [tag.text.strip() for tag in tags_container[0].find_all("span")]
                        vacancy_dict["Skills"] = all_tags
                    else:
                        vacancy_dict["Skills"] = ""

                all_vacancy_list.append(vacancy_dict)
        i+=1
    else:
        break

len(all_vacancy_list)

1796

In [4]:
df = pd.DataFrame.from_dict(all_vacancy_list)
df.to_excel('Software Vacancy.xlsx')